# Q&A
- Q. 在编译模型阶段，定义了多输出loss函数和权重，但在训练阶段，打印的loss却不等于各个loss的加权和
```
model.compile(loss=my_loss, optimizer='adam', loss_weights=[0.5, 0.5])
model.fit(dataset, epochs=2, steps_per_epoch=2, verbose=1)
```
输出：loss(21.9610) != 0.5 * 1.3583 - 0.5 * 1.5867
```
Epoch 1/2
1/2 [==============>...............] - ETA: 0s - loss: 21.9610 - dense_4_loss: 1.3583 - dense_5_loss: 1.5867
2/2 [==============================] - 0s 140ms/step - loss: 22.1960 - dense_4_loss: 1.9502 - dense_5_loss: 1.5183
Epoch 2/2
1/2 [==============>...............] - ETA: 0s - loss: 21.8526 - dense_4_loss: 1.3555 - dense_5_loss: 1.5861
2/2 [==============================] - 0s 1ms/step - loss: 22.0872 - dense_4_loss: 1.9470 - dense_5_loss: 1.5171
```

    A: 因为总的loss中包含了权重正则化损失部分：
```
def build_net(input_tensor):
    out1 = keras.layers.Dense(1, kernel_initializer='glorot_normal', activation='linear',
                              kernel_regularizer=keras.regularizers.l2(10))(input_tensor)
    out2 = keras.layers.Dense(1, kernel_initializer='glorot_normal', activation='linear',
                              kernel_regularizer=keras.regularizers.l2(10))(input_tensor)
    return [out1, out2]
```


- Q. 在将`.ckpt.index` + `.ckpt.data` 模型转为`pb`的时候，为什么还要先保存为`h5`，然后再加载模型，再保存为`pb`？

    A: 因为原来保存为`.ckpt.index` + `.ckpt.data` 的时候没有保存图信息，加载也只加载权重信息：
```
model.load_weights(latest)
...
cp_callback = ModelCheckpoint(path, save_weights_only=True, period=ckpt_period)
```
导致`keras.backend.get_session().graph.as_graph_def()`没有图结构信息。
（理论上我是构建了网络图模型，然后再加载权重的，所以应该也得有图结构信息，但实际上没有）
所以需要将模型完全保存为`h5`（包含图信息），然后重新加载进来，再保存为`pb`：
```
model.save(h5_path, overwrite=True, include_optimizer=False)
model = keras.models.load_model(h5_path)
...
graph = tf.graph_util.remove_training_nodes(sess.graph.as_graph_def())
graph_frozen = tf.graph_util.convert_variables_to_constants(sess, graph, output_names)
tf.train.write_graph(graph_frozen, pb_model_dir, pb_model_name, as_text=False)
```

- Q. 一直没办法用多GPU的模式运行？

    A: `tf.enable_eager_execution()`模型跑不了多GPU，要注释掉这句。
